## Tutorial: Stitching Networks between Chameleon Sites

This notebook demonstrates how to use ExoGENI to stitch networks between Chameleon sites. It will walk through reserving and configuring stitchable networks on each Chameleon site and using the xoStitch tool to create a layer2 circuit between the networks. 

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)

TODO: add the rest of the module links when they are ready

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [1]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project
chi.set('project_name', 'CH-816532') # Replace with your project name

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=2

#Network Config
physical_network='exogeni'
cidr='192.168.42.0/24'

## Create Chicago Network and Server

#### Create a Lease at Chicago

In [2]:
#Set the region
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
uc_lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(uc_lease, indent=2))

{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLease",
  "end_date": "2020-05-13T13:40:00.000000",
  "reservations": [
    {
      "status": "pending",
      "before_end": "default",
      "lease_id": "ef32d8fe-4966-4ce1-b1c8-428032b03151",
      "resource_id": "b8a6a17f-36c0-4802-b68b-30c4cebb8429",
      "max": 2,
      "created_at": "2020-05-12 13:40:06",
      "min": 2,
      "updated_at": "2020-05-12 13:40:12",
      "missing_resources": false,
      "hypervisor_properties": "",
      "on_start": "default",
      "resource_properties": "[\"==\", \"$node_type\", \"compute_haswell\"]",
      "id": "50cc6ca4-1619-4825-8218-371e33b4e815",
      "resource_type": "physical:host",
      "resources_changed": false
    },
    {
      "status": "pending",
      "lease_id": "ef32d8fe-4966-4ce1-b1c8-428032b03151",
      "resource_id": "005e6b4e-0775-4f01-82c2-35ed1ba2a740",
      "created_at": "2020-05-12 13:40:12",
      "updated_at": "2020-05-12 13

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [3]:
#Get the lease by name
uc_lease = get_lease_by_name(lease_name)

uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', uc_lease['reservations']))[0]['id']
uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

print("uc_compute_reservation_id: " + uc_compute_reservation_id)
print("uc_network_reservation_id: " + uc_network_reservation_id)
print("uc_floatingip_reservation_id: " + uc_floatingip_reservation_id)

uc_compute_reservation_id: 50cc6ca4-1619-4825-8218-371e33b4e815
uc_network_reservation_id: 9ebf1027-a0d4-44cb-8cda-9bce64e844b5
uc_floatingip_reservation_id: d2f6aba5-2521-402b-95b0-7ccbd2dd4364


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [5]:
chi.set('region_name', 'CHI@UC')  
#Get the network
network = get_network_by_name(network_name)
print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
uc_network_vlan = network['provider:segmentation_id']
print('uc_network_vlan: ' + str(uc_network_vlan))


{
  "provider:physical_network": "exogeni",
  "ipv6_address_scope": null,
  "revision_number": 1,
  "port_security_enabled": true,
  "mtu": 1500,
  "id": "20e66a22-423c-43eb-84c6-9d5b047847d2",
  "router:external": false,
  "availability_zone_hints": [],
  "availability_zones": [],
  "ipv4_address_scope": null,
  "shared": false,
  "project_id": "d9faac3973a847f1b718fa765fe312e2",
  "status": "ACTIVE",
  "subnets": [],
  "description": "",
  "tags": [],
  "updated_at": "2020-05-12T13:41:14Z",
  "provider:segmentation_id": 3295,
  "name": "pruthNet",
  "admin_state_up": true,
  "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
  "created_at": "2020-05-12T13:41:13Z",
  "provider:network_type": "vlan"
}
Network ID: 20e66a22-423c-43eb-84c6-9d5b047847d2
uc_network_vlan: 3295


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [6]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

{
  "subnets": [
    {
      "service_types": [],
      "description": "",
      "enable_dhcp": true,
      "tags": [],
      "network_id": "20e66a22-423c-43eb-84c6-9d5b047847d2",
      "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
      "created_at": "2020-05-12T13:41:45Z",
      "dns_nameservers": [],
      "updated_at": "2020-05-12T13:41:45Z",
      "gateway_ip": "192.168.42.1",
      "ipv6_ra_mode": null,
      "allocation_pools": [
        {
          "start": "192.168.42.2",
          "end": "192.168.42.254"
        }
      ],
      "host_routes": [],
      "revision_number": 0,
      "ip_version": 4,
      "ipv6_address_mode": null,
      "cidr": "192.168.42.0/24",
      "project_id": "d9faac3973a847f1b718fa765fe312e2",
      "id": "09ccbffd-f199-44b4-ba97-41d46de294ac",
      "subnetpool_id": null,
      "name": "pruthsubnet"
    }
  ]
}


#### Add a Router

TODO: add links here

In [7]:
router = create_router(router_name, network_name)

print(json.dumps(router, indent=2))

{
  "router": {
    "status": "ACTIVE",
    "external_gateway_info": {
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "enable_snat": true,
      "external_fixed_ips": [
        {
          "subnet_id": "c3950603-9e04-4cc5-be8d-1efbfe59fc0a",
          "ip_address": "192.5.87.29"
        }
      ]
    },
    "availability_zone_hints": [],
    "availability_zones": [],
    "description": "",
    "tags": [],
    "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
    "created_at": "2020-05-12T13:41:49Z",
    "admin_state_up": true,
    "updated_at": "2020-05-12T13:41:50Z",
    "flavor_id": null,
    "revision_number": 3,
    "routes": [],
    "project_id": "d9faac3973a847f1b718fa765fe312e2",
    "id": "2de8e297-be1b-48f2-a008-66d057b18c45",
    "name": "pruthRouter"
  }
}


#### Attach the Router and Subnet

TODO: Add links here

In [8]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

{'network_id': '20e66a22-423c-43eb-84c6-9d5b047847d2',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '09ccbffd-f199-44b4-ba97-41d46de294ac',
 'subnet_ids': ['09ccbffd-f199-44b4-ba97-41d46de294ac'],
 'port_id': '3515d18f-db62-458d-82e7-b4cb7dbc6e5f',
 'id': '2de8e297-be1b-48f2-a008-66d057b18c45'}

## Create TACC Network and Server

#### Create a Lease at TACC

In [9]:
#Set the region
chi.set('region_name', 'CHI@TACC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
tacc_lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(tacc_lease, indent=2))

{
  "status": "STARTING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLease",
  "end_date": "2020-05-13T13:41:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "036eef6b-30bd-445f-b721-d2c049f27f60",
      "resource_id": "0da16677-811f-4a04-a063-94108c522aad",
      "created_at": "2020-05-12 13:42:16",
      "updated_at": "2020-05-12 13:42:18",
      "missing_resources": false,
      "id": "6ee14cef-a737-4b10-89ae-ecf407992d30",
      "resource_type": "network",
      "resources_changed": false
    },
    {
      "status": "pending",
      "lease_id": "036eef6b-30bd-445f-b721-d2c049f27f60",
      "resource_id": "474afa18-ac00-484d-b2fb-e4fbc838bd66",
      "network_id": "6189521e-06a0-4c43-b163-16cc11ce675b",
      "created_at": "2020-05-12 13:42:18",
      "updated_at": "2020-05-12 13:42:20",
      "required_floatingips": [],
      "missing_resources": false,
      "amount": 1,
      "id": "cef4b7df-ea56-41d3-a83c-76f87f47cbb6",
  

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [10]:
#Get the lease by name
tacc_lease = get_lease_by_name(lease_name)

tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
tacc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', tacc_lease['reservations']))[0]['id']
tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

print("tacc_compute_reservation_id: " + tacc_compute_reservation_id)
print("tacc_network_reservation_id: " + tacc_network_reservation_id)
print("tacc_floatingip_reservation_id: " + tacc_floatingip_reservation_id)

tacc_compute_reservation_id: fc2f8ad3-b568-4efd-a93d-beed8ce952e9
tacc_network_reservation_id: 6ee14cef-a737-4b10-89ae-ecf407992d30
tacc_floatingip_reservation_id: cef4b7df-ea56-41d3-a83c-76f87f47cbb6


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [11]:
chi.set('region_name', 'CHI@TACC')  
#Get the network
network = get_network_by_name(network_name)
print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
tacc_network_vlan = network['provider:segmentation_id']
print('tacc_network_vlan: ' + str(tacc_network_vlan))


{
  "provider:physical_network": "exogeni",
  "ipv6_address_scope": null,
  "revision_number": 1,
  "port_security_enabled": true,
  "mtu": 1500,
  "id": "95d69043-5877-401a-8a6c-2530e8e930a2",
  "router:external": false,
  "availability_zone_hints": [],
  "availability_zones": [],
  "ipv4_address_scope": null,
  "shared": false,
  "project_id": "d9faac3973a847f1b718fa765fe312e2",
  "status": "ACTIVE",
  "subnets": [],
  "description": "",
  "tags": [],
  "updated_at": "2020-05-12T13:42:21Z",
  "provider:segmentation_id": 3505,
  "name": "pruthNet",
  "admin_state_up": true,
  "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
  "created_at": "2020-05-12T13:42:21Z",
  "provider:network_type": "vlan"
}
Network ID: 95d69043-5877-401a-8a6c-2530e8e930a2
tacc_network_vlan: 3505


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [12]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

{
  "subnets": [
    {
      "service_types": [],
      "description": "",
      "enable_dhcp": true,
      "tags": [],
      "network_id": "95d69043-5877-401a-8a6c-2530e8e930a2",
      "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
      "created_at": "2020-05-12T13:43:04Z",
      "dns_nameservers": [],
      "updated_at": "2020-05-12T13:43:04Z",
      "gateway_ip": "192.168.42.1",
      "ipv6_ra_mode": null,
      "allocation_pools": [
        {
          "start": "192.168.42.2",
          "end": "192.168.42.254"
        }
      ],
      "host_routes": [],
      "revision_number": 0,
      "ip_version": 4,
      "ipv6_address_mode": null,
      "cidr": "192.168.42.0/24",
      "project_id": "d9faac3973a847f1b718fa765fe312e2",
      "id": "363d1af3-e859-41cc-91c9-85fe25e85b2c",
      "subnetpool_id": null,
      "name": "pruthsubnet"
    }
  ]
}


#### Add a Router

TODO: add links here

In [13]:
router = create_router(router_name, network_name)

print(json.dumps(router, indent=2))

{
  "router": {
    "status": "ACTIVE",
    "external_gateway_info": {
      "network_id": "6189521e-06a0-4c43-b163-16cc11ce675b",
      "enable_snat": true,
      "external_fixed_ips": [
        {
          "subnet_id": "a58a9684-844d-4424-821a-b1e75ebfc0f7",
          "ip_address": "129.114.109.79"
        }
      ]
    },
    "availability_zone_hints": [],
    "availability_zones": [],
    "description": "",
    "tags": [],
    "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
    "created_at": "2020-05-12T13:43:06Z",
    "admin_state_up": true,
    "updated_at": "2020-05-12T13:43:07Z",
    "flavor_id": null,
    "revision_number": 3,
    "routes": [],
    "project_id": "d9faac3973a847f1b718fa765fe312e2",
    "id": "60060994-1a76-46e6-84f4-2131000f984f",
    "name": "pruthRouter"
  }
}


#### Attach the Router and Subnet

TODO: Add links here

In [14]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

{'network_id': '95d69043-5877-401a-8a6c-2530e8e930a2',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '363d1af3-e859-41cc-91c9-85fe25e85b2c',
 'subnet_ids': ['363d1af3-e859-41cc-91c9-85fe25e85b2c'],
 'port_id': 'afaee6db-dc80-4e54-b33d-c1f90ad900f4',
 'id': '60060994-1a76-46e6-84f4-2131000f984f'}

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [15]:
chi.set('region_name', 'CHI@UC')  
#create the server
server = create_server(server_name, 
                       reservation_id=uc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [17]:
chi.set('region_name', 'CHI@UC')  
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

Floating IP: 192.5.87.220


#### Start the TACC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [16]:
chi.set('region_name', 'CHI@TACC')

print("server_name " + server_name)
print("tacc_compute_reservation_id " + tacc_compute_reservation_id)
print("key_name " + key_name) 
print("network_name " + network_name)
print("image_name " + image_name)
print("flavor_name " + flavor_name)

#create the server
server = create_server(server_name, 
                       reservation_id=tacc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


server_name pruthServer
tacc_compute_reservation_id fc2f8ad3-b568-4efd-a93d-beed8ce952e9
key_name pruth-jupyter
network_name pruthNet
image_name CC-CentOS7
flavor_name baremetal


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [18]:
chi.set('region_name', 'CHI@TACC')  

floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

Floating IP: 129.114.109.124


## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [24]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

uc_vlan  3295 , tacc_vlan  3505 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem


log4j:WARN No appenders could be found for logger (org.renci.ahab.libndl.LIBNDL).
log4j:WARN Please initialize the log4j system properly.


#### Check the Status of the Circuit

In [25]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

uc_vlan  3295 , tacc_vlan  3505 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem
statusSlice: uc-3295-tacc-3505
Resource=sp2 State=Active
Resource=sp2-sp1-net State=Active
Resource=sp2 State=Active
state: STABLE


log4j:WARN No appenders could be found for logger (org.apache.commons.httpclient.HttpClient).
log4j:WARN Please initialize the log4j system properly.


## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [26]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

uc_vlan  3295 , tacc_vlan  3505 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem
deleteSlice: uc-3295-tacc-3505

log4j:WARN No appenders could be found for logger (org.apache.commons.httpclient.HttpClient).
log4j:WARN Please initialize the log4j system properly.


### Delete TACC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [32]:
chi.set('region_name', 'CHI@TACC')

In [28]:
delete_server_by_name(server_name)

()

#### De-configure Network
TODO: break up into steps

In [33]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

{'network_id': '95d69043-5877-401a-8a6c-2530e8e930a2',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '363d1af3-e859-41cc-91c9-85fe25e85b2c',
 'subnet_ids': ['363d1af3-e859-41cc-91c9-85fe25e85b2c'],
 'port_id': 'afaee6db-dc80-4e54-b33d-c1f90ad900f4',
 'id': '60060994-1a76-46e6-84f4-2131000f984f'}

In [34]:
delete_router_by_name(router_name)

()

In [35]:
delete_subnet_by_name(subnet_name)

In [36]:
delete_network_by_name(network_name)

#### Release Lease

In [37]:
delete_lease_by_name(lease_name)

Deleted lease pruthLease with id 036eef6b-30bd-445f-b721-d2c049f27f60


### Delete UC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [38]:
chi.set('region_name', 'CHI@UC')

In [31]:
delete_server_by_name(server_name)

()

#### De-configure Network
TODO: break up into steps

In [39]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

{'network_id': '20e66a22-423c-43eb-84c6-9d5b047847d2',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '09ccbffd-f199-44b4-ba97-41d46de294ac',
 'subnet_ids': ['09ccbffd-f199-44b4-ba97-41d46de294ac'],
 'port_id': '3515d18f-db62-458d-82e7-b4cb7dbc6e5f',
 'id': '2de8e297-be1b-48f2-a008-66d057b18c45'}

In [40]:
delete_router_by_name(router_name)

()

In [41]:
delete_subnet_by_name(subnet_name)

In [42]:
delete_network_by_name(network_name)

#### Release Lease

In [43]:
delete_lease_by_name(lease_name)

Deleted lease pruthLease with id ef32d8fe-4966-4ce1-b1c8-428032b03151
